In [ ]:
!pip install hyperas

In [8]:
import h5py
from os.path import join,exists
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation,Flatten, Bidirectional, LSTM
from tensorflow.keras.optimizers import RMSprop
from hyperas.distributions import choice
from hyperas import optim
from tensorflow.keras.constraints import max_norm
from random import randint
import keras
import numpy as np
from sklearn.utils import shuffle
from hyperopt import Trials, STATUS_OK, tpe
import os
import matplotlib.pyplot as plt

In [4]:
os.chdir('/content/drive/My Drive/github/NNanobody/regression')

In [30]:
def create_model(X_train, Y_train, X_test, Y_test):
    W_maxnorm = 3
    DROPOUT = {{choice([0.3,0.5,0.7])}}
    
    model = Sequential()
    model.add(Bidirectional(LSTM(units=64, return_sequences=True), input_shape=(20,1,20)))
    model.add(Bidirectional(LSTM(units=64)))
    model.add(Dense(1))

    myoptimizer = RMSprop(lr={{choice([1e-1,0.01,0.001,0.0001,1e-5])}}, rho=0.9, epsilon=1e-6)
    mylossfunc='mean_squared_error'
    model.compile(loss=mylossfunc, optimizer=myoptimizer)
    result = model.fit(X_train, Y_train, batch_size=100, epochs=20,validation_split=0.1, verbose=False)

    val_loss = np.amax(result.history['val_loss'])

    return {'loss': val_loss, 'status': STATUS_OK, 'model': model}

In [6]:
def data():
  data_train = h5py.File('./data/Hold out Top 4%/train.h5.batch1','r')
  X_train = np.array(data_train['data'])
  Y_train = np.array(data_train['label'])
  X_train_shuffled, Y_train_shuffled = shuffle(X_train, Y_train)

  data_test = h5py.File('./data/Test set Regression/test.h5.batch1', 'r')
  X_test = np.array(data_test['data'])
  Y_test = np.array(data_test['label'])
  return X_train_shuffled, Y_train_shuffled, X_test, Y_test

In [31]:
best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='seq_LSTM',
                                          verbose=False)

  0%|          | 0/10 [00:00<?, ?it/s, best loss: ?]


ValueError: ignored